In [14]:
from torch_geometric.nn import VGAE
from data_generator import ssp_data
import pickle
import torch
# from torch_geometric.datasets import Planetoid
# import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges
import os
# from torch.utils.tensorboard import SummaryWriter


In [15]:
file = open('data_pickle', 'rb')
ssp_obj = pickle.load(file)
dataset = ssp_obj.test_data
data = dataset
data.train_mask = data.val_mask = data.test_mask = data.y = None
data = train_test_split_edges(data)


class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VariationalGCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv_mu = GCNConv(2 * out_channels, out_channels, cached=True)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

c:\Users\jqcla\Documents\GitHub\Honours-Thesis\.conda\lib\site-packages\torch_geometric\deprecation.py:22: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [20]:
out_channels = 2
num_features = dataset.num_features
epochs = 4000


model = VGAE(VariationalGCNEncoder(num_features, out_channels))  # new line

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = dataset.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [21]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    
    loss = loss + (1 / data.num_nodes) * model.kl_loss()  # new line
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [22]:
for epoch in range(1, epochs + 1):
    loss = train()
    # auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, Loss: {:.4f}'.format(epoch, loss))
    # print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    
    # writer.add_scalar('auc train',auc,epoch) # new line
    # writer.add_scalar('ap train',ap,epoch)   # new line

Epoch: 001, Loss: 6602.9570
Epoch: 002, Loss: 1.9632
Epoch: 003, Loss: 1.6352
Epoch: 004, Loss: 1.7434
Epoch: 005, Loss: 1.7031
Epoch: 006, Loss: 1.8513
Epoch: 007, Loss: 1.5112
Epoch: 008, Loss: 1.6691
Epoch: 009, Loss: 1.7333
Epoch: 010, Loss: 1.6365
Epoch: 011, Loss: 1.7452
Epoch: 012, Loss: 1.5733
Epoch: 013, Loss: 1.8712
Epoch: 014, Loss: 1.7395
Epoch: 015, Loss: 1.5909
Epoch: 016, Loss: 1.7263
Epoch: 017, Loss: 1.5431
Epoch: 018, Loss: 1.8638
Epoch: 019, Loss: 1.6536
Epoch: 020, Loss: 1.6633
Epoch: 021, Loss: 1.5071
Epoch: 022, Loss: 1.5502
Epoch: 023, Loss: 1.6216
Epoch: 024, Loss: 1.5658
Epoch: 025, Loss: 1.5882
Epoch: 026, Loss: 1.5708
Epoch: 027, Loss: 1.6514
Epoch: 028, Loss: 1.5135
Epoch: 029, Loss: 1.6627
Epoch: 030, Loss: 1.4748
Epoch: 031, Loss: 1.5042
Epoch: 032, Loss: 1.8877
Epoch: 033, Loss: 1.5788
Epoch: 034, Loss: 1.4834
Epoch: 035, Loss: 1.5212
Epoch: 036, Loss: 1.4827
Epoch: 037, Loss: 1.5747
Epoch: 038, Loss: 1.5201
Epoch: 039, Loss: 1.5627
Epoch: 040, Loss: 1.54